## __init_subclass__를 사용해서 클래스 확장


In [ ]:
#기본적인 파이썬 객체를 JSON 문자열로 변환

import json

class Serializable:
    def __init__(self, *args):
        self.args = args

    def serialize(self):
        return json.dumps({'args': self.args})

#위의 클래스를 사용하면 간단한 불변 데이터 구조를 쉽게 직렬화

class Point2D(Serializable):
    def __init__(self,x,y):
        super().__init__(x,y)
        self.x = x
        self.y = y

    def __repr__(self):
        return f'Point2D({self.x}, {self.y})'

point = Point2D(5,3)


In [ ]:
#Serialzable를 부모 클래스로 하고, 이를 활용하여 데이터 역직렬화

class Deserializable(Serializable):
    @classmethod
    def deserialize(cls, json_data):
        params = json.loads(json_data)
        return cls(*params['args'])

class BetterPoint2D(Deserializable):
    ....

before = BetterPoint2D(5,3)
print('이전:', before)
data = before.serialize()
print('직렬화한 값:', data)
after = BetterPoint2D.deserialize(data)
print('이후:', after)

In [ ]:
#공통 함수를 만들어서 객체의 클래스 이름 직렬화하여 JSON 데이터 포함

class BetterSerializable:
    def __init__(self, *args):
        self.args = args

    def serialize(self):
        return json.dumps({
            'class': self.__class__.__name__,
            'args': self.args,
        })

    def __repr__(self):
        name = self.__class__.__name__
        args_str = ', '.json(str(x) for x in self.args)
        return f'{name}({args_str})'

In [ ]:
#클래스 이름을 객체 생성자로 매핑 가능
#매핑 사용시 deserialize함수는 register_class를 통해 등록된 모든 클래스 작동

registry = {}

def register_class(target_class):
    registry[target_class.__name__] = target_class

def deserialize(data):
    params = json.loads(data)
    name = params['class']
    target_class = registry[name]
    return target_class(*params['args'])


    

하위 클래스의 호출문을 잊어먹게 되어 제대로 기능을 사용할 수 없는 구조이다.(위의 코드 방식)

In [ ]:
#메타 클래스 활용

class Meta(type):
    def __new__(meta, name, bases, class_dict):
        cls = type.__new__(meta, name, bases, class_dict)
        register_class(cls)
        return cls

class RegisteredSerializable(BetterSerializable, metaclass = Meta):
    pass

In [ ]:
#잘 동작하는지 확인
class Vector3D(RegisteredSerializable):
    def __init__(self, x,y,z):
        super().__init__(x,y,z)
        self.x, self.y, self.z = x,y,z

In [ ]:
#__init_subclass__를 특별 메서드로 활용하면 코드의 잡음이 줄어든다.

class BetterRegisteredSerializable(BetterSerializable):
    def __init_subclass__(cls):
        super().__init_subclass__()
        register_class(cls)

class Vector1D(BetterRegisteredSerializable):
    def __init__(self, magnitude):
        super().__init__(magnitude)
        self.magnitude = magnitude

**위와 같이 코드를 구성할 시, 클래스 등록에 __init_subclass__ 혹은 메타 클래스를 사용하면, 상속 트리가 제대로 돼 있는 한 클래스 등록을 잊어버릴 일이 없다고 보장이 되고 위의방식은 직렬화일 때, 객체-관계 매핑, 확장성 플러그인 시스템, 콜백 훅엗 작동

### Summary

- 클래스 등록은 파이썬 프로그램을 모듈화할 때 유용한 패턴

- 메타클래스를 사용하면, 프로그램 안에서 기반 클래스를 상속한 하위 클래스가 정의될 때마다 등록 코드를 자동으로 실행

- 메타클래스를 등록에 사용하면 클래스 등록 함수를 호출하지 않아서 생기는 오류를 피할 수 있다

- 표준적인 메타클래스 방식보다는  __init_subclass__가 더 낫다. __init_subclass__쪽이 더 깔끔하고 이해하기 쉬움
